In [3]:
import warnings
warnings.filterwarnings('ignore', message = "Conversions")
import os
import sys
import re
import pandas as pd
import numpy as np
import geopandas as gpd
import pymongo
import requests
import json
import shapely.geometry
from pymongo import MongoClient, GEOSPHERE
sys.path.append('../src')
import code1_data_ as f1

/usr/local/Caskroom/miniconda/base/envs/ironhack/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [4]:
client = MongoClient()
compis = client.ironhack.compi

## Requirements for location (part I)

### Filtered documents from "compi" collection 

In [5]:
q = {"deadpooled_year":None,
     "total_money_raised": {"$ne":"$0","$not":{"$regex":"k"}},
     "offices.country_code":{"$in":
                             ['CZE','DEU','DNK',
                              'ESP','FRA','GBR',
                              'HUN','IRL','ITA',
                              'NLD','SWE','CHN','USA'
                             ],
                             "$ne":None},
     "offices.latitude":{"$ne":None},
     "offices.longitude":{"$ne":None},
     "number_of_employees":{"$gte":100}
    }

pj = {"name":1,
      "_id":0,
      "category_code":1,
      "offices.city":1,
      "offices.country_code":1,
      "offices.latitude":1,
      "offices.longitude":1
     }

res = compis.find(q,pj).sort("offices.country_code",1)
res1 = list(res)

In [6]:
df1 = pd.DataFrame(res1)
df1.head(3)

,name,category_code,offices
0,99designs,design,"[{'city': 'San Francisco', 'country_code': 'US..."
1,Netsize,mobile,"[{'city': 'Meudon (Paris)', 'country_code': '..."
2,CPM Braxis,consulting,"[{'city': 'SÃ£o Paulo', 'country_code': 'BRA',..."


In [7]:
df2 = df1.explode('offices')
df2.reset_index(inplace = True, drop = True)
df2.head(3)

,name,category_code,offices
0,99designs,design,"{'city': 'San Francisco', 'country_code': 'USA..."
1,99designs,design,"{'city': 'Collingwood', 'country_code': 'AUS',..."
2,99designs,design,"{'city': 'Berlin', 'country_code': 'DEU', 'lat..."


In [8]:
df2.offices[0]

{'city': 'San Francisco',
 'country_code': 'USA',
 'latitude': 37.7955307,
 'longitude': -122.4005983}

In [9]:
df2_unnested = pd.json_normalize(df2.offices, max_level = 1)

In [10]:
df3 = pd.concat([df2, df2_unnested], axis=1)
df3.head()

,name,category_code,offices,city,country_code,latitude,longitude
0,99designs,design,"{'city': 'San Francisco', 'country_code': 'USA...",San Francisco,USA,37.795531,-122.400598
1,99designs,design,"{'city': 'Collingwood', 'country_code': 'AUS',...",Collingwood,AUS,-37.802659,144.986855
2,99designs,design,"{'city': 'Berlin', 'country_code': 'DEU', 'lat...",Berlin,DEU,52.498620,13.446903
3,Netsize,mobile,"{'city': 'Meudon (Paris)', 'country_code': 'F...",Meudon (Paris),FRA,48.894312,2.288319
4,Netsize,mobile,"{'city': 'West Chatswood', 'country_code': 'AU...",West Chatswood,AUS,-33.867139,151.207114


<div class="girk">
### converting to geopandas dataframe with shapely point objects</div><i class="fa fa-lightbulb-o "></i>

In [2]:
gdf = gpd.GeoDataFrame(df3, geometry = gpd.points_from_xy(df3.longitude, df3.latitude))

NameError: name 'gpd' is not defined

<div class="girk">
### converting shapely point objects to GeoJSON point objects</div><i class="fa fa-lightbulb-o "></i>

In [13]:
gdf['geometry']=gdf['geometry'].apply(lambda x: shapely.geometry.mapping([x]))

AttributeError: 'list' object has no attribute '__geo_interface__'

In [11]:
gdf.head(3)

,name,category_code,offices,city,country_code,latitude,longitude,geometry
0,99designs,design,"{'city': 'San Francisco', 'country_code': 'USA...",San Francisco,USA,37.795531,-122.400598,"{'type': 'Point', 'coordinates': (-122.4005983..."
1,99designs,design,"{'city': 'Collingwood', 'country_code': 'AUS',...",Collingwood,AUS,-37.802659,144.986855,"{'type': 'Point', 'coordinates': (144.9868546,..."
2,99designs,design,"{'city': 'Berlin', 'country_code': 'DEU', 'lat...",Berlin,DEU,52.498620,13.446903,"{'type': 'Point', 'coordinates': (13.4469031, ..."


### removing unnecessary column with duplicated info and sorting data by country_code

In [11]:
df3.drop(columns="offices", inplace=True)

In [12]:
df3.sort_values(by = ['country_code'], inplace = True, ignore_index = True)

<div class="girk">
### Creating new collectio<span class="mark">n "companies</span>1" in ironhack database and 2dsphere index</div><i class="fa fa-lightbulb-o "></i>

In [16]:
companies1 = client.ironhack.companies1
companies1.create_index([("geometry",GEOSPHERE)])

'geometry_2dsphere'

<div class="mark">
### Converting geoDF into a list of dictionaries to insert filtered documents with added GeoJSON information to "companies1" collection.</div><i class="fa fa-lightbulb-o "></i>

In [14]:
df3

,name,category_code,city,country_code,latitude,longitude
0,99designs,design,Collingwood,AUS,-37.802659,144.986855
1,Netsize,mobile,West Chatswood,AUS,-33.867139,151.207114
2,CPM Braxis,consulting,SÃ£o Paulo,BRA,-23.591042,-46.687202
3,Adknowledge,advertising,Toronto,CAN,43.647455,-79.395374
4,Novariant,other,Santiago,CHL,-29.959641,-71.255956
...,...,...,...,...,...,...
310,Acquia,enterprise,Burlington,USA,42.524527,-71.137784
311,Etsy,ecommerce,Brooklyn,USA,40.694731,-73.983268
312,Tesla Motors,automotive,Palo Alto,USA,37.496737,-122.245323
313,fabrik,network_hosting,San Mateo,USA,37.553743,-122.300246


In [17]:
f1.export_df_to_csv(df3)

Attention: Export succesfull
Attention: File exported correctly to '../data_processed/dfcompanies1_v1.csv'


<div class="girk">
No hecho</div><i class="fa fa-lightbulb-o "></i>

In [42]:
gdf_dic = gdf.to_dict('records')

In [43]:
companies1.insert_many(gdf_dic)

### Accessing relevant information on the lcoation of each office in the database to create a 2dsphere index.

In [ ]:
tok1 = os.getenv("tok1")
tok2 = os.getenv("tok2")

### Comparing number of companies that meet criteria in major non-European countries

### Comparing number of companies that meet criteria in European countries